In [1]:
from seaborn import load_dataset
from sklearn.model_selection import StratifiedKFold
from junifer.storage import HDF5FeatureStorage
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nilearn import plotting

In [2]:
from julearn import run_cross_validation
from julearn import PipelineCreator
from julearn.utils import configure_logging
from sklearn.model_selection import StratifiedKFold
from julearn.viz import plot_scores
from junifer.storage import HDF5FeatureStorage
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
storage = HDF5FeatureStorage("./test.hdf5")
df_fc = storage.read_df('BOLD_parccortical-TianxS3x3TxMNI6thgeneration')
df_alff = storage.read_df('BOLD_ALFF_parccortical-TianxS3x3TxMNI6thgeneration')
df_participants = pd.read_csv("./participants.tsv", sep="\t")
df_participants.rename(columns={"participant_id": "subject"}, inplace=True)
df_participants.set_index("subject", inplace=True)
parcellation_labels = df_alff.columns.values

In [12]:
df_fc

,HIP-head-m-rh~HIP-head-m-rh,HIP-head-l-rh~HIP-head-m-rh,HIP-head-l-rh~HIP-head-l-rh,HIP-body-rh~HIP-head-m-rh,HIP-body-rh~HIP-head-l-rh,HIP-body-rh~HIP-body-rh,HIP-tail-rh~HIP-head-m-rh,HIP-tail-rh~HIP-head-l-rh,HIP-tail-rh~HIP-body-rh,HIP-tail-rh~HIP-tail-rh,...,aGP-lh~CAU-DA-lh,aGP-lh~CAU-body-lh,aGP-lh~CAU-tail-lh,aGP-lh~lAMY-lh,aGP-lh~mAMY-lh,aGP-lh~THA-DP-lh,aGP-lh~NAc-shell-lh,aGP-lh~NAc-core-lh,aGP-lh~pGP-lh,aGP-lh~aGP-lh
subject,,,,,,,,,,,,,,,,,,,,,
sub-60037,1.0,0.445415,1.0,0.422543,0.629345,1.0,0.373190,0.403897,0.815198,1.0,...,-0.050286,0.453025,0.255845,0.207602,0.311071,0.319423,0.075310,0.470268,0.668813,1.0
sub-50050,1.0,0.622711,1.0,0.057205,0.056285,1.0,0.304999,0.197752,0.100615,1.0,...,0.253765,0.299922,0.330320,0.256886,0.071310,-0.090398,0.161196,0.096315,0.062020,1.0
sub-60052,1.0,0.013398,1.0,0.282062,-0.250302,1.0,0.078152,-0.090185,0.414318,1.0,...,0.695390,0.607813,-0.138061,0.740099,-0.036907,0.672284,-0.515414,0.629850,-0.921649,1.0
sub-70072,1.0,-0.101317,1.0,-0.010458,0.940979,1.0,0.448583,0.489166,0.595634,1.0,...,0.522609,-0.538351,0.542082,0.602157,-0.438970,0.057261,0.346975,0.322756,-0.169879,1.0
sub-60077,1.0,0.590232,1.0,-0.098065,0.093687,1.0,0.311375,0.277318,-0.037015,1.0,...,0.430066,0.211432,0.234703,0.179742,0.239331,-0.121815,0.262073,0.008747,0.462076,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-10274,1.0,0.687619,1.0,0.265365,0.237683,1.0,-0.220626,-0.443597,0.088793,1.0,...,0.140926,0.229983,0.117708,-0.119184,0.386444,0.014178,0.314083,0.247088,0.313814,1.0
sub-60006,1.0,0.545006,1.0,0.068902,0.425815,1.0,-0.104555,-0.010870,0.421374,1.0,...,0.051795,0.048991,-0.215124,0.137471,0.187409,0.208283,0.256838,0.141140,0.260661,1.0
sub-11044,1.0,0.509289,1.0,0.364605,0.412933,1.0,0.148764,0.342515,0.392104,1.0,...,0.050915,-0.279489,0.462264,0.006837,0.355860,0.355011,-0.013650,0.508114,0.496179,1.0


In [4]:
df_alff_full = df_alff.merge(df_participants, on="subject")
df_fc_full = df_fc.merge(df_participants, on="subject")

In [5]:
es_control = lambda x: 1 if x =="CONTROL" else 0
df_fc_full["is_control"] = df_fc_full["diagnosis"].apply(es_control)
df_alff_full["is_control"] = df_alff_full["diagnosis"].apply(es_control)

In [16]:
df_fc_dev, df_fc_test = train_test_split(df_fc_full, test_size=0.2, stratify=df_fc_full["is_control"])
df_alff_dev, df_alff_test = df_alff_full.loc[df_fc_dev.index], df_alff_full.loc[df_fc_test.index]

x_fc_cols, x_alff_cols, target = list(df_fc.columns), list(df_alff.columns),  "is_control" 

In [11]:
df_fc.columns

Index(['HIP-head-m-rh~HIP-head-m-rh', 'HIP-head-l-rh~HIP-head-m-rh',
       'HIP-head-l-rh~HIP-head-l-rh', 'HIP-body-rh~HIP-head-m-rh',
       'HIP-body-rh~HIP-head-l-rh', 'HIP-body-rh~HIP-body-rh',
       'HIP-tail-rh~HIP-head-m-rh', 'HIP-tail-rh~HIP-head-l-rh',
       'HIP-tail-rh~HIP-body-rh', 'HIP-tail-rh~HIP-tail-rh',
       ...
       'aGP-lh~CAU-DA-lh', 'aGP-lh~CAU-body-lh', 'aGP-lh~CAU-tail-lh',
       'aGP-lh~lAMY-lh', 'aGP-lh~mAMY-lh', 'aGP-lh~THA-DP-lh',
       'aGP-lh~NAc-shell-lh', 'aGP-lh~NAc-core-lh', 'aGP-lh~pGP-lh',
       'aGP-lh~aGP-lh'],
      dtype='object', length=1275)

In [ ]:
def plot_correlation_matrix(lower_tri_array, parcellation, title="Correlation Matrix"):
    n = int((np.sqrt(1 + 8 * len(lower_tri_array)) - 1) / 2)
    
    corr_matrix = np.zeros((n, n))
    
    tri_indices = np.tril_indices(n)
    corr_matrix[tri_indices] = lower_tri_array

    plt.figure(figsize=(10, 8))
    plotting.plot_matrix(corr_matrix, figure=(10, 8), labels=parcellation, colorbar=True, vmax=0.8, vmin=-0.8, title=title)
    plt.show()

# Get the mean value of the correlation matrix for all subjects with diagnosis AD
lower_mean_control = df_fc_full[df_fc_full['diagnosis'] == 'CONTROL'][df_fc.columns].mean().values
lower_mean_adhd = df_fc_full[df_fc_full['diagnosis'] == 'ADHD'][df_fc.columns].mean().values
lower_mean_schz = df_fc_full[df_fc_full['diagnosis'] == 'SCHZ'][df_fc.columns].mean().values

plot_correlation_matrix(lower_mean_control, parcellation=parcellation_labels, title="Mean Correlation Matrix for Control Subjects")
plot_correlation_matrix(lower_mean_adhd, parcellation=parcellation_labels, title="Mean Correlation Matrix for ADHD Subjects")
plot_correlation_matrix(lower_mean_schz, parcellation=parcellation_labels, title="Mean Correlation Matrix for SCHZ Subjects")


In [13]:
from sklearn.decomposition import PCA

In [14]:
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 42)

In [17]:
X_fc_dev = df_fc_dev
pca = PCA(n_components=100)
X_fc_dev_pca = pca.fit_transform(X_fc_dev[x_fc_cols])

In [20]:
df_fc_dev_pca = pd.DataFrame(X_fc_dev_pca, columns = list([str(x) for x in range(X_fc_dev_pca.shape[1])]), index=df_fc_dev.index)
df_fc_dev_pca_cols = list(df_fc_dev_pca.columns)
df_fc_dev_pca[target] = df_fc_dev[target].values

In [44]:
creator1 = PipelineCreator(problem_type="classification")
#creator1.add("zscore")
creator1.add("rf", n_estimators=[25,50,100,200], max_depth=[None, 5, 10, 20])

In [29]:
df_fc_dev_pca.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '91', '92', '93', '94', '95', '96', '97', '98', '99', 'is_control'],
      dtype='object', length=101)

In [54]:
scores, model, inspector = run_cross_validation(
    X= x_alff_cols,
    y=target,
    data = df_alff_dev,
    model=creator1,
    cv=cv,
    return_train_score=True,
    return_estimator="all",
    search_params={"cv":StratifiedKFold(n_splits=5)},
    return_inspector=True,
    scoring = "roc_auc"
)

The following columns are not defined in X_types: ['HIP-head-m-rh', 'HIP-head-l-rh', 'HIP-body-rh', 'HIP-tail-rh', 'THA-VPm-rh', 'THA-VPl-rh', 'THA-VAi-rh', 'THA-VAs-rh', 'THA-DAm-rh', 'THA-DAl-rh', 'PUT-VA-rh', 'PUT-DA-rh', 'PUT-VP-rh', 'PUT-DP-rh', 'CAU-VA-rh', 'CAU-DA-rh', 'CAU-body-rh', 'CAU-tail-rh', 'lAMY-rh', 'mAMY-rh', 'THA-DP-rh', 'NAc-shell-rh', 'NAc-core-rh', 'pGP-rh', 'aGP-rh', 'HIP-head-m-lh', 'HIP-head-l-lh', 'HIP-body-lh', 'HIP-tail-lh', 'THA-VPm-lh', 'THA-VPl-lh', 'THA-VAi-lh', 'THA-VAs-lh', 'THA-DAm-lh', 'THA-DAl-lh', 'PUT-VA-lh', 'PUT-DA-lh', 'PUT-VP-lh', 'PUT-DP-lh', 'CAU-VA-lh', 'CAU-DA-lh', 'CAU-body-lh', 'CAU-tail-lh', 'lAMY-lh', 'mAMY-lh', 'THA-DP-lh', 'NAc-shell-lh', 'NAc-core-lh', 'pGP-lh', 'aGP-lh']. They will be treated as continuous.
/home/nyanovsky/miniconda3/envs/neurodc/lib/python3.11/site-packages/julearn/prepare.py:503: RuntimeWarning: The following columns are not defined in X_types: ['HIP-head-m-rh', 'HIP-head-l-rh', 'HIP-body-rh', 'HIP-tail-rh', 'THA

In [55]:
scores

,fit_time,score_time,estimator,test_score,train_score,n_train,n_test,repeat,fold,cv_mdsum
0,10.429352,0.005238,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.589827,1.0,171,43,0,0,4bbff3df81cf8854822a1b557f37a710
1,10.833624,0.013769,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.560870,1.0,171,43,0,1,4bbff3df81cf8854822a1b557f37a710
2,10.549078,0.004918,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.600000,1.0,171,43,0,2,4bbff3df81cf8854822a1b557f37a710
3,10.636809,0.004516,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.619565,1.0,171,43,0,3,4bbff3df81cf8854822a1b557f37a710
4,10.881558,0.009613,"GridSearchCV(cv=StratifiedKFold(n_splits=5, ra...",0.475000,1.0,172,42,0,4,4bbff3df81cf8854822a1b557f37a710


## Modelo